# **Introducción al Análisis Exploratorio de Datos (EDA) del Proyecto "China Garden"**

En un entorno competitivo y dinámico como el mercado de restaurantes en Estados Unidos, el análisis de datos se ha convertido en una herramienta esencial para entender la posición de una marca, anticipar tendencias y tomar decisiones estratégicas fundamentadas. En este proyecto, nuestro objetivo es realizar un análisis exploratorio de datos (EDA) para una reconocida cadena de restaurantes chinos, **China Garden**, con el fin de apoyar su estrategia de expansión en la costa este del país.

El EDA proporcionará una base sólida para identificar oportunidades de crecimiento y optimización operativa, analizando tanto datos de desempeño actual como información sobre la competencia. Este análisis también permitirá evaluar factores clave como la satisfacción del cliente, la presencia de mercado, y las ubicaciones óptimas para nuevos restaurantes. 

#### **Objetivos del Análisis Exploratorio de Datos**

Para lograr los objetivos generales del proyecto, nuestro EDA se centrará en los siguientes puntos:

1. **Revisión de Tipos de Datos y Descripción de Datasets:**
   - Examinar los tipos de datos por columna en cada dataset disponible para asegurar su adecuación al análisis posterior. Se incluirá una descripción detallada de cada dataset, abordando su propósito y relevancia en el contexto del proyecto.

2. **Justificación de la Eliminación de Columnas:**
   - Analizar y explicar la eliminación de columnas que no aportan valor al análisis o que presentan datos inconsistentes. Esta etapa es fundamental para mejorar la eficiencia del procesamiento de datos y mantener la integridad del análisis.

3. **Identificación y Manejo de Valores Nulos:**
   - Detectar los valores nulos en los datasets y proponer estrategias adecuadas de manejo, como la imputación de valores basados en la media, mediana o la eliminación de registros, dependiendo de la relevancia y el impacto de los datos faltantes.

4. **Detección y Tratamiento de Registros Duplicados:**
   - Identificar registros duplicados que puedan sesgar el análisis. Proponer métodos para su tratamiento, como la eliminación de duplicados exactos o el análisis de duplicados parciales para decidir la mejor acción.

5. **Análisis Gráfico y Visualización de Datos:**
   - Utilizar herramientas como **Matplotlib** y **Seaborn** para realizar análisis gráficos que permitan entender el comportamiento de los datos. Este enfoque incluirá, pero no se limitará a:
     - Visualización de la distribución de restaurantes y hoteles en la costa este.
     - Cálculo del porcentaje de mercado capturado por los restaurantes de comida china en comparación con otros tipos de restaurantes.
     - Identificación de los 10 principales restaurantes de comida china en términos de cantidad, promedio de calificaciones y presencia de mercado.
     - Análisis geográfico para entender la ubicación de los restaurantes de "China Garden" y su competencia directa.
     - Análisis de las zonas con mayor concentración de hoteles para identificar las áreas de mayor interés turístico.
     - Cualquier otro análisis pertinente que pueda ofrecer insights sobre el comportamiento del mercado y las oportunidades de expansión.

6. **Exportacion a Parquet**

Este EDA servirá como el cimiento sobre el cual se basarán todos los análisis futuros, ayudando a identificar patrones y tendencias que orientarán las decisiones estratégicas de "China Garden". A través de un enfoque minucioso y detallado, buscaremos proporcionar un conocimiento profundo y detallado del mercado, permitiendo a la marca mejorar su posicionamiento y maximizar su potencial de crecimiento en la costa este de Estados Unidos.

### **Empecemos**

## **1. Carga de datos Google maps**

**Descripción de los Datos de Google Maps**

Para este proyecto, se han recopilado datos provenientes de Google Maps, una de las plataformas más utilizadas por los consumidores para compartir opiniones sobre sus experiencias en restaurantes. Estos datos se encuentran organizados en dos carpetas principales, cada una con una estructura específica que facilita el análisis detallado de la información.

**1. Carpeta: Metadata Sitios**

La carpeta Metadata Sitios contiene información relevante sobre los establecimientos de interés, en este caso, los restaurantes de comida china en la costa este de Estados Unidos. Esta carpeta incluye un total de 11 archivos JSON que deberán ser unificados en un solo archivo consolidado para facilitar su análisis. Cada archivo JSON en esta carpeta proporciona datos específicos sobre las características de los restaurantes, como su nombre, ubicación geográfica, tipo de cocina, horarios de atención, y otros detalles relevantes que permiten identificar y comparar a cada establecimiento dentro del mercado.

**2. Carpeta: Review-Estados**

La carpeta Review-Estados contiene una colección organizada de reseñas de clientes, separadas por estado, en formato JSON. Dentro de esta carpeta, cada subcarpeta representa un estado de la costa este de Estados Unidos, y a su vez, cada subcarpeta contiene múltiples archivos JSON que recogen las opiniones y calificaciones de los usuarios sobre los restaurantes. Estos archivos de reseñas capturan datos cruciales como la fecha de la reseña, la calificación otorgada, los comentarios detallados de los clientes, y otros atributos relevantes como el número de "me gusta" que ha recibido cada reseña.

**BIBLIOTECAS UTILIZADAS**

In [2]:
import pandas as pd
import re
import glob
import os
import pyarrow as pa
import pyarrow.parquet as pq
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords

### **1.1 Metadata-Sitios**

Se cargan los archivos .json y se concatenan en un mismo Dataframe

In [2]:
# Ruta del directorio que contiene los archivos JSON
ruta_directorio = './Datasets/Google/metadata-sitios/'

# Usamos glob para encontrar todos los archivos .json en el directorio
archivos_json = glob.glob(os.path.join(ruta_directorio, '*.json'))

# Lista para almacenar los DataFrames de cada archivo
dfs = []

# Iteramos sobre cada archivo JSON encontrado
for archivo in archivos_json:
    # Cargamos el archivo JSON en un DataFrame
    df_temp = pd.read_json(archivo, lines=True)
    
    # Agregamos el DataFrame a la lista
    dfs.append(df_temp)

# Combinamos todos los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)

# Mostramos el tamaño del DataFrame combinado
print(f"Tamaño del DataFrame Metadata-Sitios combinado: {df_final.shape}")

Tamaño del DataFrame Metadata-Sitios combinado: (3025011, 15)


#### **Eliminar columnas**

Debido a que estos tipos de datos no son necesarios para nuestro análisis, se eliminarán las siguientes columnas: 'description', 'MISC', 'state', 'relative_results', 'url'.

In [3]:
# Eliminamos las columnas especificadas del DataFrame
columnas_a_eliminar = ['description', 'MISC', 'state', 'relative_results', 'url']
df_final = df_final.drop(columns=columnas_a_eliminar)

In [4]:
# Verificamos el resultado mostrando las primeras filas
df_final.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun..."
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,[Textile exporter],4.5,6,None,None
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],..."


De acuerdo a la dirección en la columna "address", se crea la Columna "estado", que nos da el estado del local comercial. Se hace porque nuestro dataset contiene todos los comercios del país, y el análisis será solo sobre la costa Este del mismo. 

In [5]:

# Usamos una expresión regular para extraer el estado (dos letras antes del código postal)
df_final['estado'] = df_final['address'].str.extract(r',\s*([A-Z]{2})\s*\d{5}')

# Verificamos que la nueva columna 'estado' se ha creado correctamente
df_final[['address', 'estado']].head()

,address,estado
0,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",GA
1,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",CA
2,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",CA
3,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",CA
4,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",CA


Se eliminan las filas de los comercios que no están en la costa este del país.

In [6]:
# Lista de los estados que queremos conservar de la costa este
estados_deseados = [
    'ME',  # Maine
    'NH',  # New Hampshire
    'MA',  # Massachusetts
    'RI',  # Rhode Island
    'CT',  # Connecticut
    'NY',  # New York
    'NJ',  # New Jersey
    'DE',  # Delaware
    'MD',  # Maryland
    'VA',  # Virginia
    'NC',  # North Carolina
    'SC',  # South Carolina
    'GA',  # Georgia
    'FL'   # Florida
]

In [7]:

# Filtramos el DataFrame para que solo queden las filas con los estados deseados
df_metadata_ce = df_final[df_final['estado'].isin(estados_deseados)]

# Verificamos que el filtrado se haya realizado correctamente
print(df_metadata_ce[['address', 'estado']].head())
print(f"Tamaño del DataFrame después del filtrado: {df_metadata_ce.shape}")

                                              address estado
0   Porter Pharmacy, 129 N Second St, Cochran, GA ...     GA
14  NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...     FL
45  Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...     FL
56  Seminole Family Restaurant, 6864 Seminole Blvd...     FL
58  A-Top Insurance, 1009 Brighton Beach Ave, Broo...     NY
Tamaño del DataFrame después del filtrado: (946582, 11)


In [8]:
df_metadata_ce.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL


Se agrega una nueva columna con el nombre completo del estado

In [9]:
# Diccionario de mapeo de abreviaturas a nombres completos de estados
mapa_estados = {
    'ME': 'Maine',
    'NH': 'New Hampshire',
    'MA': 'Massachusetts',
    'RI': 'Rhode Island',
    'CT': 'Connecticut',
    'NY': 'New York',
    'NJ': 'New Jersey',
    'DE': 'Delaware',
    'MD': 'Maryland',
    'VA': 'Virginia',
    'NC': 'North Carolina',
    'SC': 'South Carolina',
    'GA': 'Georgia',
    'FL': 'Florida'
}

# Creamos la nueva columna 'nombre_estado' usando el mapeo
df_metadata_ce['nombre_estado'] = df_metadata_ce['estado'].map(mapa_estados)

# Verificamos que la nueva columna se haya creado correctamente
print(df_metadata_ce[['estado', 'nombre_estado']].head())


   estado nombre_estado
0      GA       Georgia
14     FL       Florida
45     FL       Florida
56     FL       Florida
58     NY      New York


C:\Users\54370\AppData\Local\Temp\ipykernel_10024\2111716646.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata_ce['nombre_estado'] = df_metadata_ce['estado'].map(mapa_estados)


In [10]:
# Mostramos el tamaño del DataFrame combinado
print(f"Tamaño del DataFrame combinado: {df_metadata_ce.shape}")

Tamaño del DataFrame combinado: (946582, 12)


Se exporta el dataframe a parquet para que su tamaño sea menor y su  velocidad de procesamiento sea mayor. 

In [11]:
# Guardamos el DataFrame con la nueva columna en un archivo Parquet
df_metadata_ce.to_parquet('Datasets/Google/metadata-sitios/metadata-sitios-costaeste.parquet')


In [12]:
df_metadata_ce.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado,nombre_estado
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA,Georgia
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL,Florida
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL,Florida


Se carga el parquet porque la velocidad de procesamiento aumentará de manera significativa y para posteriormente realizar la limpieza de datos que serán utlizados.


In [13]:
# Especificamos la ruta de tu archivo Parquet
ruta_archivo = 'Datasets/Google/metadata-sitios/metadata-sitios-costaeste.parquet'

# Cargamos el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo)

# Mostramos las primeras filas del DataFrame
df.head(3)


,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado,nombre_estado
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA,Georgia
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL,Florida
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL,Florida


In [14]:
df

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado,nombre_estado
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA,Georgia
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL,Florida
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL,Florida
56,Seminole Family Restaurant,"Seminole Family Restaurant, 6864 Seminole Blvd...",0x88c2fb8614e57ce9:0x40b51628875ad6e9,27.834885,-82.787419,None,3.6,8,$$,None,FL,Florida
58,A-Top Insurance,"A-Top Insurance, 1009 Brighton Beach Ave, Broo...",0x89c24469c758686b:0x641f5b84cb9bedfa,40.578254,-73.959127,"[Insurance broker, Insurance agency]",2.0,4,None,"[[Thursday, 10AM–6PM], [Friday, 10AM–6PM], [Sa...",NY,New York
...,...,...,...,...,...,...,...,...,...,...,...,...
3024967,SVS Vision Optical Centers,"SVS Vision Optical Centers, 1551 Niagara Falls...",0x89d3721399f576df:0xe572aaa0d5ae28fc,42.997026,-78.821580,"[Eye care center, Contact lenses supplier, Opt...",4.7,68,None,"[[Saturday, 9AM–1PM], [Sunday, Closed], [Monda...",NY,New York
3024968,Express Liquor,"Express Liquor, 6589 Thompson Rd suite b, Syra...",0x89d9f2a96ec09509:0xc8d86190079d033,43.088776,-76.092804,[Liquor store],3.5,8,None,"[[Saturday, 10AM–10PM], [Sunday, 12–6PM], [Mon...",NY,New York
3024969,Allstar Collision and Paint,"Allstar Collision and Paint, 6050 Sheridan Dr,...",0x89d3744be13dc5f5:0x22f97178c788cb06,42.978248,-78.707804,"[Auto body shop, Auto dent removal service, Ca...",5.0,48,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",NY,New York
3024976,House of Gourmet -- 食全食美,"House of Gourmet -- 食全食美, 2865 Sheridan Dr Sui...",0x89d373386b718fff:0xf82c90ef48e7b23,42.980251,-78.827337,"[Chinese restaurant, Sichuan restaurant]",4.3,58,None,"[[Saturday, 11:15AM–9:45PM], [Sunday, 11:15AM–...",NY,New York


Se crea la columna ciudad de acuerdo a la columna "address" para ser utlizada posteriormente en el análisis de datos.

In [15]:

# Utilizamos una expresión regular para extraer la ciudad, que se encuentra entre la dirección y el estado
df['city'] = df['address'].str.extract(r',\s*([^,]+),\s*[A-Z]{2}\s*\d{5}')

# Mostramos las primeras filas para verificar
df[['address', 'city']].head()

,address,city
0,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Cochran
14,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",Miami
45,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",Pinellas Park
56,"Seminole Family Restaurant, 6864 Seminole Blvd...",Seminole
58,"A-Top Insurance, 1009 Brighton Beach Ave, Broo...",Brooklyn


In [16]:
df

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado,nombre_estado,city
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA,Georgia,Cochran
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL,Florida,Miami
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL,Florida,Pinellas Park
56,Seminole Family Restaurant,"Seminole Family Restaurant, 6864 Seminole Blvd...",0x88c2fb8614e57ce9:0x40b51628875ad6e9,27.834885,-82.787419,None,3.6,8,$$,None,FL,Florida,Seminole
58,A-Top Insurance,"A-Top Insurance, 1009 Brighton Beach Ave, Broo...",0x89c24469c758686b:0x641f5b84cb9bedfa,40.578254,-73.959127,"[Insurance broker, Insurance agency]",2.0,4,None,"[[Thursday, 10AM–6PM], [Friday, 10AM–6PM], [Sa...",NY,New York,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024967,SVS Vision Optical Centers,"SVS Vision Optical Centers, 1551 Niagara Falls...",0x89d3721399f576df:0xe572aaa0d5ae28fc,42.997026,-78.821580,"[Eye care center, Contact lenses supplier, Opt...",4.7,68,None,"[[Saturday, 9AM–1PM], [Sunday, Closed], [Monda...",NY,New York,Buffalo
3024968,Express Liquor,"Express Liquor, 6589 Thompson Rd suite b, Syra...",0x89d9f2a96ec09509:0xc8d86190079d033,43.088776,-76.092804,[Liquor store],3.5,8,None,"[[Saturday, 10AM–10PM], [Sunday, 12–6PM], [Mon...",NY,New York,Syracuse
3024969,Allstar Collision and Paint,"Allstar Collision and Paint, 6050 Sheridan Dr,...",0x89d3744be13dc5f5:0x22f97178c788cb06,42.978248,-78.707804,"[Auto body shop, Auto dent removal service, Ca...",5.0,48,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",NY,New York,Buffalo
3024976,House of Gourmet -- 食全食美,"House of Gourmet -- 食全食美, 2865 Sheridan Dr Sui...",0x89d373386b718fff:0xf82c90ef48e7b23,42.980251,-78.827337,"[Chinese restaurant, Sichuan restaurant]",4.3,58,None,"[[Saturday, 11:15AM–9:45PM], [Sunday, 11:15AM–...",NY,New York,Tonawanda


In [17]:
# Guardamos el DataFrame con la nueva columna en un archivo Parquet
df.to_parquet('Datasets/Google/metadata-sitios/metadata-sitios-costaeste-estado-ciudad.parquet')

### **Creación de los archivos parquet de Reviews-Estados de Google maps**

In [18]:
# Función para procesar la carga de datos
def procesar_archivos_json(ruta_directorio, nombre_estado, columnas_a_eliminar=['pics', 'resp']):
    # Usamos glob para encontrar todos los archivos .json en el directorio
    archivos_json = glob.glob(os.path.join(ruta_directorio, '*.json'))
    
    # Lista para almacenar los DataFrames de cada archivo
    dfs = []

    # Iteramos sobre cada archivo JSON encontrado
    for archivo in archivos_json:
        # Cargamos el archivo JSON en un DataFrame
        df_temp = pd.read_json(archivo, lines=True)
        
        # Agregamos el DataFrame a la lista
        dfs.append(df_temp)

    # Combinamos todos los DataFrames en uno solo
    df_combinado = pd.concat(dfs, ignore_index=True)

    # Mostramos el tamaño del DataFrame combinado
    print(f"Tamaño del DataFrame combinado para {nombre_estado}: {df_combinado.shape}")

    # Eliminamos las columnas especificadas del DataFrame
    df_combinado = df_combinado.drop(columns=columnas_a_eliminar)

    # Guardamos el DataFrame en un archivo Parquet
    ruta_salida = f'Datasets/Google/review-estados/review-estados-parquet/review-{nombre_estado.lower()}.parquet'
    df_combinado.to_parquet(ruta_salida)
    
    print(f"Archivo Parquet guardado en: {ruta_salida}")
    
    return df_combinado

**01. Estado de la Florida**

In [19]:
df_florida = procesar_archivos_json('./Datasets/Google/review-estados/review-Florida', 'Florida')


Tamaño del DataFrame combinado para Florida: (2850000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-florida.parquet


**02. Estado de Georgia**

In [20]:
df_georgia = procesar_archivos_json('./Datasets/Google/review-estados/review-Georgia', 'Georgia')

Tamaño del DataFrame combinado para Georgia: (1950000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-georgia.parquet


**03. Estado de North Carolina**

In [21]:
df_north_carolina = procesar_archivos_json('./Datasets/Google/review-estados/review-North_Carolina', 'North_Carolina')

Tamaño del DataFrame combinado para North_Carolina: (2250000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-north_carolina.parquet


**04. Estado de South Carolina**

In [22]:
df_south_carolina = procesar_archivos_json('./Datasets/Google/review-estados/review-South_Carolina', 'South_Carolina')

Tamaño del DataFrame combinado para South_Carolina: (2100000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-south_carolina.parquet


**05. Estado de Maine**

In [23]:
df_maine = procesar_archivos_json('./Datasets/Google/review-estados/review-Maine', 'Maine')

Tamaño del DataFrame combinado para Maine: (1123881, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-maine.parquet


**06. Estado de Massachusetts**

In [24]:
df_maine = procesar_archivos_json('./Datasets/Google/review-estados/review-Massachusetts', 'Massachusetts')

Tamaño del DataFrame combinado para Massachusetts: (2400000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-massachusetts.parquet


**07. Estado de New Hampshire**

In [25]:
df_new_hampshire = procesar_archivos_json('./Datasets/Google/review-estados/review-New_Hampshire', 'New_Hampshire')

Tamaño del DataFrame combinado para New_Hampshire: (1296603, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-new_hampshire.parquet


**08.Estado de Rhode Island**

In [26]:
df_rhode_island = procesar_archivos_json('./Datasets/Google/review-estados/review-Rhode_Island', 'Rhode_Island')

Tamaño del DataFrame combinado para Rhode_Island: (890006, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-rhode_island.parquet


**09. Estado de New York**

In [27]:
df_new_york = procesar_archivos_json('./Datasets/Google/review-estados/review-New_York', 'New_York')

Tamaño del DataFrame combinado para New_York: (2700000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-new_york.parquet


**10. Estado de Connecticut**

In [28]:
df_connecticut = procesar_archivos_json('./Datasets/Google/review-estados/review-Connecticut', 'Connecticut')

Tamaño del DataFrame combinado para Connecticut: (2680107, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-connecticut.parquet


**11. Estado de Delaware**


In [29]:
df_delaware = procesar_archivos_json('./Datasets/Google/review-estados/review-Delaware', 'Delaware')

Tamaño del DataFrame combinado para Delaware: (905537, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-delaware.parquet


**12. Estado de New Jersey**

In [30]:
df_new_jersey = procesar_archivos_json('./Datasets/Google/review-estados/review-New_Jersey', 'New_Jersey')

Tamaño del DataFrame combinado para New_Jersey: (1950000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-new_jersey.parquet


**13. Estado de Maryland**

In [31]:
df_maryland = procesar_archivos_json('./Datasets/Google/review-estados/review-Maryland', 'Maryland')

Tamaño del DataFrame combinado para Maryland: (2400000, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-maryland.parquet


**14. Estado de Virginia**

In [32]:
df_virginia = procesar_archivos_json('./Datasets/Google/review-estados/review-Virginia', 'Virginia')

Tamaño del DataFrame combinado para Virginia: (1662059, 8)
Archivo Parquet guardado en: Datasets/Google/review-estados/review-estados-parquet/review-virginia.parquet


### **2. Carga de datos Yelp**

**Descripción de los Datos de Yelp**

Para el análisis de la cadena de restaurantes China Garden en la costa este de Estados Unidos, se han recopilado datos detallados de Yelp, una de las plataformas de reseñas de restaurantes más importantes. Estos datos se encuentran distribuidos en cinco archivos distintos, cada uno de los cuales proporciona información relevante para entender el comportamiento del mercado y la percepción de los consumidores.

**1. Archivo: business.pkl**

El archivo business.pkl contiene información esencial sobre los negocios listados en Yelp, específicamente sobre los restaurantes de interés. Este archivo proporciona datos detallados de cada establecimiento, incluyendo su nombre, dirección, categoría, atributos adicionales (como si ofrece opciones vegetarianas o si tiene estacionamiento), y su ubicación geográfica exacta. Estos datos son fundamentales para realizar un análisis geoespacial, identificar patrones en las ubicaciones de los restaurantes, y comparar características de los competidores dentro del mismo mercado.

**2. Archivo: Checkin.json**

El archivo Checkin.json proporciona los datos de horarios de reservas y registros de visitas de los usuarios a los restaurantes. Este archivo permite analizar las horas pico de actividad, la popularidad de los restaurantes en diferentes días de la semana, y la frecuencia de visitas de los clientes. Este análisis es crucial para entender los patrones de tráfico y comportamiento de los consumidores, lo que puede ayudar a optimizar los horarios de operación y estrategias de marketing.

**3. Archivo: Review.json**

El archivo Review.json contiene las reseñas de los clientes que han visitado los restaurantes de interés. Cada reseña incluye información valiosa como la calificación otorgada (de 1 a 5 estrellas), el texto de la reseña, la fecha en que se escribió, y otros atributos que reflejan la experiencia del cliente. Este archivo es fundamental para realizar un análisis de sentimientos y evaluar las opiniones de los consumidores sobre China Garden y sus competidores. Además, permite identificar temas recurrentes en los comentarios de los clientes, puntos fuertes, áreas de mejora, y tendencias emergentes en la percepción del mercado.

**4. Archivo: Tip**

El archivo Tip contiene una recopilación de recomendaciones o consejos breves que los usuarios han dejado sobre los restaurantes en Yelp. Aunque estos "tips" son más cortos que las reseñas tradicionales, a menudo proporcionan insights rápidos y directos sobre lo que los clientes consideran relevante o digno de destacar. Este archivo permite complementar el análisis de reseñas con datos adicionales que pueden ayudar a identificar las expectativas y preferencias de los clientes.

**5. Archivo: User**

El archivo User proporciona información sobre los usuarios de Yelp que han interactuado con los restaurantes de la cadena y sus competidores. Incluye detalles como el número total de reseñas realizadas por cada usuario, su ubicación, el promedio de calificaciones otorgadas, y otros comportamientos del usuario en la plataforma. Este archivo es clave para identificar clientes leales, entender los perfiles demográficos y de comportamiento de los clientes, y determinar cómo interactúan diferentes grupos de consumidores con los restaurantes de interés.

**Proceso de Preparación de los Dato**s
Para realizar un análisis integral, se procederá con la limpieza y unificación de los datos de cada archivo:

Integración de la información de negocios del archivo business.pkl con los datos de los archivos de reseñas, tips, check-ins y usuarios.
Procesamiento de los archivos JSON para extraer insights relevantes sobre la actividad de los clientes, patrones de reserva, y calificaciones.
Análisis combinado de los datos de Yelp para identificar oportunidades de mejora y estrategias de crecimiento para "China Garden" en la costa este de Estados Unidos.
Esta estructura de datos permitirá llevar a cabo un análisis exhaustivo que considere no solo la competencia y el posicionamiento actual de "China Garden", sino también las expectativas y percepciones de los clientes en tiempo real.

### **01. Carga y exportación a parquet del archivo Bussiness.pkl**

In [33]:
# Ruta al archivo .pkl
ruta_archivo = 'Datasets/Yelp/business.pkl'

# Cargamos el archivo .pkl en un DataFrame
df = pd.read_pickle(ruta_archivo)

# Mostramos las primeras filas del DataFrame para verificar que se haya cargado correctamente
df.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Exploración de columnas
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [35]:
df.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

Se encontraron varias columnas duplicadas

In [36]:
# Eliminamos columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

# Mostramos las columnas después de eliminar duplicados
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [37]:
df.shape

(150346, 14)

In [38]:
# Eliminamos las columnas especificadas del DataFrame
columnas_a_eliminar = ['address', 'is_open', 'attributes', 'hours']
df = df.drop(columns=columnas_a_eliminar)

df.shape

(150346, 10)

**Se filtra el DataFrame por estados de la costa este**

In [39]:
# Lista de estados de la costa este que queremos mantener
east_coast_states = ['ME', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'DE', 'MD', 'VA', 'NC', 'SC', 'GA', 'FL']

# Filtramos el DataFrame para eliminar los valores nulos en la columna 'state' 
# y mantenemos solo los estados de la costa este
df_filtrado = df[df['state'].isin(east_coast_states) & df['state'].notna()]

# Mostramos el DataFrame filtrado
df_filtrado.head(3)


,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,categories
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,"Department Stores, Shopping, Fashion"
13,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,"General Dentistry, Dentists, Health & Medical,..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,Largo,FL,33771,27.916116,-82.760461,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants"


In [40]:
df_filtrado.shape

(37133, 10)

**Se agrega una columna con el nombre de estado completo**

In [41]:
# Diccionario de códigos de estado a nombres completos
state_names = {
    'ME': 'Maine',
    'NH': 'New Hampshire',
    'MA': 'Massachusetts',
    'RI': 'Rhode Island',
    'CT': 'Connecticut',
    'NY': 'New York',
    'NJ': 'New Jersey',
    'DE': 'Delaware',
    'MD': 'Maryland',
    'VA': 'Virginia',
    'NC': 'North Carolina',
    'SC': 'South Carolina',
    'GA': 'Georgia',
    'FL': 'Florida'
}

# Creamos la nueva columna 'nombre_estado' utilizando el diccionario y el método .map()
df_filtrado['nombre_estado'] = df_filtrado['state'].map(state_names)

# Mostramos el DataFrame con la nueva columna
df_filtrado.head(3)


C:\Users\54370\AppData\Local\Temp\ipykernel_10024\2195659317.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['nombre_estado'] = df_filtrado['state'].map(state_names)


,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,categories,nombre_estado
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,"Department Stores, Shopping, Fashion",Florida
13,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,"General Dentistry, Dentists, Health & Medical,...",Florida
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,Largo,FL,33771,27.916116,-82.760461,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants",Florida


In [42]:
# Guardamos el DataFrame en un archivo Parquet
df_filtrado.to_parquet('Datasets/Yelp/parquet_files/Business.parquet')

### **02. Carga y exportación a parquet del archivo Checkin.json**

In [43]:
#Ruta de tu archivo JSON 1
ruta_archivo = 'Datasets\Yelp\checkin.json'

# Cargamos el archivo JSON en un DataFrame de pandas usando lines=True
df = pd.read_json(ruta_archivo, lines=True)

# Mostramos el DataFrame
df.head(3)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"


In [44]:
# Guardamos el DataFrame en un archivo Parquet
df.to_parquet('Datasets/Yelp/parquet_files/Checkin.parquet')

### **03. Carga y exportación a parquet del archivo review.json**

In [45]:
# Carga review.json
# Intentamos con utf-8
try:
    with open('Datasets/Yelp/review.json', 'r', encoding='utf-8') as f:
        df_review = pd.DataFrame([json.loads(line) for line in f])
except UnicodeDecodeError:
    # Si utf-8 falla, intentamos con utf-8-sig
    with open('Datasets/Yelp/review.json', 'r', encoding='utf-8-sig') as f:
        df_review = pd.DataFrame([json.loads(line) for line in f])

In [46]:
df_review.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


In [47]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    object 
 1   user_id      object 
 2   business_id  object 
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 480.0+ MB


In [48]:
# Eliminamos columnas innecesarias
df_review.drop(['useful', 'funny', 'cool'], axis=1, inplace=True)

In [49]:
# Exportamos a parquet
df_review.to_parquet('Datasets/Yelp/parquet_files/review-yelp.parquet', index=False)

ArrowMemoryError: realloc of size 704643072 failed

#### **Se abre nuevamente business.parquet para mergear los 14 estados en el DataFrame "review"**

In [27]:
business = pd.read_parquet("Datasets/Yelp/parquet_files/Business.parquet")
review = pd.read_parquet('Datasets/Yelp/parquet_files/review-yelp.parquet')

In [28]:
merged_df = pd.merge(business, review, on='business_id', how='inner')

In [29]:
merged_df.shape

(1734757, 16)

In [30]:
merged_df.head(3)

,business_id,name,city,state,postal_code,latitude,longitude,stars_x,review_count,categories,nombre_estado,review_id,user_id,stars_y,text,date
0,UJsufbvfyfONHeWdvAHKjA,Marshalls,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,"Department Stores, Shopping, Fashion",Florida,MKf1btP1IWejTptigCLrSw,u4Dzi_282Zw6hKqZJOhsSw,5.0,What's not to love about Marshalls?! I absolut...,2020-11-20 19:02:54
1,UJsufbvfyfONHeWdvAHKjA,Marshalls,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,"Department Stores, Shopping, Fashion",Florida,MxpvjOXWjyp3xPA6qJenKw,qjfMBIZpQT9DDtw_BWCopQ,4.0,"This is a nice Marshalls, and it is pretty new...",2015-07-24 00:31:29
2,UJsufbvfyfONHeWdvAHKjA,Marshalls,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,"Department Stores, Shopping, Fashion",Florida,6dy2iBob-3lNlGVfdzXH_Q,F-OwSluUWMn0vMp3bPWMnQ,3.0,I Have not been here in so many years that I f...,2019-10-18 17:43:01


#### **Se exporta a parquet reviews-yelp**

In [32]:
merged_df.to_parquet("Datasets/Yelp/parquet_files/review-yelp.parquet")

### **Limpieza metadata_google**

In [3]:
df_metadata_google = pd.read_parquet('Datasets/Google/metadata-sitios/metadata-sitios-costaeste-estado-ciudad.parquet')
df_metadata_google.head(3)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,estado,nombre_estado,city
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",GA,Georgia,Cochran
14,NTK OUTDOOR,"NTK OUTDOOR, 2315 NW 107th Ave #1B18, Miami, F...",0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,35,None,"[[Thursday, 9AM–6PM], [Friday, 9AM–6PM], [Satu...",FL,Florida,Miami
45,Cruises Inc. - Connie Stewart,"Cruises Inc. - Connie Stewart, 6602 52nd Ln, P...",0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,2,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",FL,Florida,Pinellas Park


In [4]:
df_metadata_google.info()

<class 'pandas.core.frame.DataFrame'>
Index: 946582 entries, 0 to 3024991
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            946582 non-null  object 
 1   address         946582 non-null  object 
 2   gmap_id         946582 non-null  object 
 3   latitude        946582 non-null  float64
 4   longitude       946582 non-null  float64
 5   category        941305 non-null  object 
 6   avg_rating      946582 non-null  float64
 7   num_of_reviews  946582 non-null  int64  
 8   price           91985 non-null   object 
 9   hours           710387 non-null  object 
 10  estado          946582 non-null  object 
 11  nombre_estado   946582 non-null  object 
 12  city            945910 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 101.1+ MB


In [5]:
# Chequeo de duplicados en gmap_id
df_metadata_google.duplicated('gmap_id').sum()

np.int64(8345)

In [6]:
duplicados = df_metadata_google[df_metadata_google.duplicated(subset='gmap_id', keep=False)]
conteo = df_metadata_google['gmap_id'].value_counts()
repetidos = conteo[conteo > 1]

print(repetidos)


gmap_id
0x88f16e41928ff687:0x883dad4fd048e8f8    12
0x884d0db9065bd815:0xae8443b860eebe8e     2
0x89e841975461fb8f:0x16df15574a663326     2
0x88f5993c9c7382fd:0xfd57cf3b5826d3b9     2
0x88d9beb4fe0532c1:0xef0555c169299d6      2
                                         ..
0x88d9a4e66e40edf1:0xebbab577da8076fe     2
0x88d9aa92d831c45b:0x9b14ae9ded185345     2
0x88d9a67325994b17:0xdac6d8d4600fc268     2
0x88d9b937c540e7d7:0xc36f9d9593ca9008     2
0x4cb29c6a0460a84b:0x243771cbbefd58d9     2
Name: count, Length: 8335, dtype: int64


In [7]:
# Eliminamos duplicados del DF columna gmap_id
df_metadata_google = df_metadata_google.drop_duplicates(subset=['gmap_id'])

In [8]:
# Chequeo de duplicados en gmap_id
df_metadata_google.duplicated('gmap_id').sum()

np.int64(0)

In [9]:
# Chequeo de datos nulos en el DF
df_metadata_google.isnull().sum()

name                   0
address                0
gmap_id                0
latitude               0
longitude              0
category            5212
avg_rating             0
num_of_reviews         0
price             846698
hours             233791
estado                 0
nombre_estado          0
city                 668
dtype: int64

#### **Se borran las columnas "price" y "hours"**

In [10]:
# Eliminamos las columnas especificadas del DataFrame
columnas_a_eliminar = ['price', 'hours']
df_metadata_google = df_metadata_google.drop(columns=columnas_a_eliminar)

#### **Se eliminan filas con categorías nulas**

In [11]:
df_metadata_google= df_metadata_google.dropna(subset=['category'])

Columna city contiene 667 nulos... ubicar por latitud y longitud ???

In [52]:
# Filtrar las filas con valores nulos en la columna 'city'
filas_con_city_nulo = df_metadata_google[df_metadata_google['city'].isnull()]

# Contar las ocurrencias de cada categoría para estas filas
categorias_con_city_nulo = filas_con_city_nulo['category'].value_counts()

# Imprimir los resultados
print("Categorías de las filas con valores nulos en 'city':")
for categoria, cantidad in categorias_con_city_nulo.items():
    print(f"Categoría: {categoria}, Cantidad de nulos: {cantidad}")

# Imprimir el total de filas con 'city' nulo
print(f"\nTotal de filas con 'city' nulo: {len(filas_con_city_nulo)}")

Categorías de las filas con valores nulos en 'city':
Categoría: ['Lake'], Cantidad de nulos: 276
Categoría: ['Wetland'], Cantidad de nulos: 197
Categoría: ['Train station'], Cantidad de nulos: 67
Categoría: ['River'], Cantidad de nulos: 37
Categoría: ['Canal'], Cantidad de nulos: 12
Categoría: ['Bus stop'], Cantidad de nulos: 10
Categoría: ['Beach'], Cantidad de nulos: 10
Categoría: ['Pond'], Cantidad de nulos: 8
Categoría: ['Island'], Cantidad de nulos: 8
Categoría: ['Reservoir'], Cantidad de nulos: 6
Categoría: ['Ferry terminal'], Cantidad de nulos: 5
Categoría: ['Peninsula'], Cantidad de nulos: 3
Categoría: ['Lodging'], Cantidad de nulos: 3
Categoría: ['Park'], Cantidad de nulos: 2
Categoría: ['Bus station'], Cantidad de nulos: 2
Categoría: ['Spring'], Cantidad de nulos: 2
Categoría: ['Water'], Cantidad de nulos: 2
Categoría: ['Beach pavillion'], Cantidad de nulos: 2
Categoría: ['Beach volleyball court'], Cantidad de nulos: 1
Categoría: ['Light rail station'], Cantidad de nulos: 1
C

In [12]:
# Exportamos metadata-sitios limpio a parquet
df_metadata_google.to_parquet("Datasets/Google/metadata-sitios/metadata-limpio.parquet")

In [13]:
df_metadata_google.shape

(933025, 11)

### **Se unifica "review-estados" en un solo Dataframe**

In [14]:
# Directorio donde se encuentran tus archivos .parquet
directorio = "Datasets/Google/review-estados/review-estados-parquet"

# Listamos todos los archivos .parquet en el directorio
archivos_parquet = glob.glob(os.path.join(directorio, "*.parquet"))

# Leemos y concatenamos los archivos
df_review_estados = pd.concat([pd.read_parquet(archivo) for archivo in archivos_parquet], ignore_index=True)

In [15]:
df_review_estados.head(3)

,user_id,name,time,rating,text,gmap_id
0,1.140765e+20,Jeff Mccomish,1612059976820,5,Great friendly independent packie,0x89e64b04d274c9d9:0x84d833dcc10ae21d
1,1.015203e+20,Outlaw PiRu,1544752859553,5,Great Atmosphere and Great Service. Fantastic ...,0x89e64b04d274c9d9:0x84d833dcc10ae21d
2,1.118002e+20,Hector Acevedo,1564367962353,5,Nice neighborhood store with great sandwiches ...,0x89e64b04d274c9d9:0x84d833dcc10ae21d


In [16]:
df_review_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27158193 entries, 0 to 27158192
Data columns (total 6 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   gmap_id  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ GB


### **Chequeo Duplicados**

In [17]:
# Contar el número de duplicados
num_duplicados = df_review_estados.duplicated().sum()

# Guardar el número original de filas
num_filas_original = len(df_review_estados)

# Eliminar duplicados
df_review_estados_sin_duplicados = df_review_estados.drop_duplicates()

# Calcular el número de filas después de eliminar duplicados
num_filas_final = len(df_review_estados_sin_duplicados)

# Calcular el porcentaje de reducción
porcentaje_reduccion = ((num_filas_original - num_filas_final) / num_filas_original) * 100

# Imprimir resultados
print(f"Número de duplicados eliminados: {num_duplicados}")
print(f"Tamaño original del DataFrame: {num_filas_original} filas")
print(f"Tamaño del DataFrame sin duplicados: {num_filas_final} filas")
print(f"Porcentaje de reducción: {porcentaje_reduccion:.2f}%")

# Actualizar el DataFrame original sin duplicados
df_review_estados = df_review_estados_sin_duplicados.reset_index(drop=True)

Número de duplicados eliminados: 491681
Tamaño original del DataFrame: 27158193 filas
Tamaño del DataFrame sin duplicados: 26666512 filas
Porcentaje de reducción: 1.81%
